In [26]:
import requests
import pandas as pd
import time
import json
import os
from glob import glob
from requests.adapters import HTTPAdapter, Retry

In [27]:
path_anos = "anos/"

In [28]:
#Leitura do json com todos os modelos
arquivos = glob(os.path.join(path_anos, "*.json"))
ind_df = (pd.read_json(f) for f in arquivos)
df_anos = pd.concat(ind_df, ignore_index=True)

In [29]:
df_anos.head()

,nome,codigo,codigo_marca,codigo_modelo
0,1995 Gasolina,1995-1,10,258
1,1994 Gasolina,1994-1,10,258
2,1993 Gasolina,1993-1,10,258
3,1992 Gasolina,1992-1,10,258
4,1991 Gasolina,1991-1,10,258


In [30]:
#Cria diretório caso não exista
path_valores = "valores"

if not os.path.isdir(path_valores):
    os.makedirs(path_valores)

In [31]:
s = requests.Session()

retries = Retry(total=5,
                backoff_factor=0.5,
                status_forcelist=[ 500, 502, 503, 504 ])

s.mount('https://', HTTPAdapter(max_retries=retries))

In [33]:
#Listando os valores já coletados
lista_valores = os.listdir('valores')
lista_valores

for linha in df_anos.index:
    
    #Obtém os códigos necessários
    codigo_marca = df_anos.loc[linha, "codigo_marca"]
    codigo_modelo = df_anos.loc[linha, "codigo_modelo"]
    codigo_ano = df_anos.loc[linha, "codigo"]

    #Define a url e arquivo
    url_valores = f"https://parallelum.com.br/fipe/api/v1/carros/marcas/{codigo_marca}/modelos/{codigo_modelo}/anos/{codigo_ano}"
    nome_arquivo = f"valor_marca_{codigo_marca}_modelo_{codigo_modelo}_ano_{codigo_ano}.json"

    #Requisita se o json não está coletado
    if nome_arquivo not in lista_valores:

        #Realiza requisição
        valores = s.get(url_valores, headers={"User-Agent": "XY"})
        valores = valores.json()

        #Cria dict com as chaves utilizadas
        dict_codigo_marca_modelo_ano = {"codigo_marca": codigo_marca, "codigo_modelo": codigo_modelo, "codigo_ano": codigo_ano}

        #Atualiza dict
        valores.update(dict_codigo_marca_modelo_ano)

        #Converte o dicionário para json
        valores_json = json.dumps(valores, default=str)

        #Salva o arquivo json e fecha a instância aberta
        json_file = open(f"{path_valores}/{nome_arquivo}", "w")
        json_file.write(valores_json)
        json_file.close()

        #print("Arquivo {} salvo com sucesso.".format(nome_arquivo))

        time.sleep(0.65)
    

Arquivo valor_marca_13_modelo_285_ano_2005-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_286_ano_2002-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_286_ano_2001-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_286_ano_2000-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_286_ano_1999-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_287_ano_1993-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_287_ano_1992-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_288_ano_1993-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_288_ano_1992-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_289_ano_2011-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_289_ano_2010-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_289_ano_2009-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_289_ano_2008-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_289_ano_2007-1.json salvo com sucesso.
Arquivo valor_marca_13_modelo_289_

KeyboardInterrupt: 